# Insper - Supercomputação - Prova Intermediária

Questão sobre GPU / THRUST.


**Questão 4: Normalização de Vetor com Thrust**

Na análise de dados e no processamento de sinais, normalizar um vetor é uma operação comum que ajusta a escala dos elementos para que o vetor tenha uma norma unitária (ou seja, a soma dos quadrados dos elementos é igual a 1). Essa tarefa se torna computacionalmente intensiva para vetores de grande dimensão, fazendo com que a paralelização em GPU seja uma solução atrativa.

**Objetivo**:

Complemente o código abaixo usando a biblioteca Thrust para normalizar um vetor grande. O código inicial cria um vetor com valores aleatórios. Você deve implementar as etapas para:

1. calcular a norma L2 do vetor,
2. dividir cada elemento do vetor por essa norma e
3. imprimir o resultado final.

**Código Pré-Pronto**:

Complete o código com a sua solução.

In [4]:
%%writefile gpu.cu

#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/transform.h>
#include <thrust/functional.h>
#include <thrust/iterator/constant_iterator.h>
#include <iostream>
#include <cmath>
#include <random> // Para std::default_random_engine e std::uniform_real_distribution

// Função para gerar números aleatórios
struct prg
{
    float a, b;

    __host__ __device__
    prg(float _a = 0.f, float _b = 1.f) : a(_a), b(_b) {};

    __host__ __device__
    float operator()(const unsigned int n) const
    {
#if defined(__CUDA_ARCH__)
        // Código para execução na GPU
        unsigned int seed = n; // Define a semente com base no índice
        unsigned int value = seed * 1103515245 + 12345; // Linear congruential generator
        return a + (value % 10000) / 10000.0f * (b - a);
#else
        // Código para execução na CPU
        static std::default_random_engine rng;
        std::uniform_real_distribution<float> dist(a, b);
        rng.seed(n);
        return dist(rng);
#endif
    }
};

// Estrutura para calcular o quadrado de um elemento
struct square
{
    __host__ __device__
    float operator()(const float& x) const
    {
        return x * x;
    }
};

// Estrutura para normalizar os elementos do vetor
struct normalize
{
    float norm;

    normalize(float _norm) : norm(_norm) {}

    __host__ __device__
    float operator()(const float& x) const
    {
        return x / norm;
    }
};

int main() {
    const int N = 1000000; // Tamanho do vetor
    thrust::counting_iterator<unsigned int> index_sequence_begin(0);
    thrust::host_vector<float> h_vec(N);

    // Preencher o vetor com números aleatórios entre 1.0 e 2.0
    thrust::transform(index_sequence_begin,
                      index_sequence_begin + N,
                      h_vec.begin(),
                      prg(1.f, 2.f));

    // Imprime alguns valores do vetor criado para verificação
    for (int i = 0; i < 20; i++) {
        std::cout << "Elemento original " << i << ": " << h_vec[i] << std::endl;
    }

    // Copia o vetor do host para o dispositivo
    thrust::device_vector<float> d_vec = h_vec;

    // Calcula a norma L2 do vetor
    float norm = std::sqrt(
        thrust::transform_reduce(d_vec.begin(), d_vec.end(), square(), 0.0f, thrust::plus<float>())
    );

    // Imprime a norma calculada
    std::cout << "Norma calculada: " << norm << std::endl;

    // Normalize o vetor dividindo cada elemento pela norma L2
    thrust::transform(d_vec.begin(), d_vec.end(), d_vec.begin(), normalize(norm));

    // Copia o vetor normalizado de volta para o host
    thrust::copy(d_vec.begin(), d_vec.end(), h_vec.begin());

    // Imprime alguns valores do vetor normalizado para verificação
    for (int i = 0; i < 20; i++) {
        std::cout << "Elemento normalizado " << i << ": " << h_vec[i] << std::endl;
    }

    return 0;
}



Overwriting gpu.cu


Compilando o código

In [5]:
!nvcc -arch=sm_75 -std=c++14 gpu.cu -o gpu

Exemplo de execução:

In [6]:
%%time
!./gpu

Elemento original 0: 1.00001
Elemento original 1: 1.00001
Elemento original 2: 1.00002
Elemento original 3: 1.00002
Elemento original 4: 1.00003
Elemento original 5: 1.00004
Elemento original 6: 1.00005
Elemento original 7: 1.00005
Elemento original 8: 1.00006
Elemento original 9: 1.00007
Elemento original 10: 1.00008
Elemento original 11: 1.00009
Elemento original 12: 1.00009
Elemento original 13: 1.0001
Elemento original 14: 1.00011
Elemento original 15: 1.00012
Elemento original 16: 1.00013
Elemento original 17: 1.00013
Elemento original 18: 1.00014
Elemento original 19: 1.00015
Norma calculada: 1517.84
Elemento normalizado 0: 0.000658836
Elemento normalizado 1: 0.000658836
Elemento normalizado 2: 0.000658841
Elemento normalizado 3: 0.000658846
Elemento normalizado 4: 0.000658851
Elemento normalizado 5: 0.000658856
Elemento normalizado 6: 0.000658862
Elemento normalizado 7: 0.000658867
Elemento normalizado 8: 0.000658872
Elemento normalizado 9: 0.000658877
Elemento normalizado 10: 0